# On regarde les performances d'un algo qui prédit toujours la moyenne.

In [32]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset
from surprise import SVD
from surprise import SVD, NMF, KNNWithMeans, CoClustering
import surprise
from surprise.model_selection import train_test_split
import time
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold

## Dataset movie_100k

In [26]:
data = pd.read_csv('dataset_movie_100k.csv')[['user_id','book_id','rating']]

In [27]:
a=time.time()
eam=0
l=list(data['rating'])
lg=len(l)
l=np.array(l)
moy=l.mean()
for x in range(lg):
    eam+=abs(moy-l[x])
    
print("EAM = ", eam/lg)
print("EQM = ", l.var())
print("Temps : ",time.time()-a , "secondes.")

EAM =  0.8498001974245242
EQM =  1.1194884644316638
Temps :  0.12865638732910156 secondes.


## precison et recall

In [28]:
reader = Reader()
data = Dataset.load_from_df(data, reader)

trainset, testset = train_test_split(data, test_size=.25)

In [29]:
algo = SVD(n_factors=1, n_epochs=1,lr_all=.01, reg_all=.01) #on l'utilise juste pour avaoir la bonne forme des résultats

In [30]:
# code source à peu près présent dans la documentation de surprise
def precision_recall_at_k(predictions, k=10, threshold=4,moyenne=3.543):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((moyenne, true_r))   #on met moyenne au lieu de est 

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        l=[moyenne for j in range(len(user_ratings[:k]))]
        n_rec_k = sum((est >= threshold) for est in l)
        #on a fait un cgt ici aussi

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (moyenne >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


def F_score(pr,rc):
    return 2*(pr*rc)/(rc+pr)

### K-fold (ici 5-fold)

In [31]:
kf = KFold(n_splits=5)
indice_fold = 1

res_precision = []
res_recall= []
f_score=[]

for trainset, testset in kf.split(data):
    print('='*50)
    print ('FOLD # ' + str(indice_fold))
    print('='*50)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

    avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
    f=F_score(avg_precision,avg_recall)
    
    print (avg_precision)
    print (avg_recall)
    print(f)
    
    res_precision.append(avg_precision)
    res_recall.append(avg_recall)
    f_score.append(f)
    indice_fold += 1
    
    
print('='*50)
print ('MOYENNES')
print('='*50)
print('précision@k moyenne: ' + str(np.mean(res_precision)))
print('rappel@k moyen: ' + str(np.mean(res_recall)))
print('F-score moyen: ' + str(np.mean(f_score)))


FOLD # 1
1.0
0.020895522388059702
0.04093567251461988
FOLD # 2
1.0
0.03278688524590164
0.06349206349206349
FOLD # 3
1.0
0.014925373134328358
0.029411764705882353
FOLD # 4
1.0
0.03134328358208955
0.06078147612156295
FOLD # 5
1.0
0.03134328358208955
0.06078147612156295
MOYENNES
précision@k moyenne: 1.0
rappel@k moyen: 0.02625886958649376
F-score moyen: 0.051080490591138325


## Dataset movie_1M

In [33]:
data = pd.read_csv('dataset_movie_1M.csv')[['user_id','book_id','rating']]

In [34]:
a=time.time()
eam=0
l=list(data['rating'])
lg=len(l)
l=np.array(l)
moy=l.mean()
for x in range(lg):
    eam+=abs(moy-l[x])
    
print("EAM = ", eam/lg)
print("EQM = ", l.var())
print("Temps : ",time.time()-a , "secondes.")

EAM =  0.9338608628723692
EQM =  1.2479152852805657
Temps :  1.5150177478790283 secondes.


## precison et recall

In [35]:
reader = Reader()
data = Dataset.load_from_df(data, reader)

trainset, testset = train_test_split(data, test_size=.25)

In [36]:
algo = SVD(n_factors=1, n_epochs=1,lr_all=.01, reg_all=.01) #on l'utilise juste pour avaoir la bonne forme des résultats

In [37]:
kf = KFold(n_splits=5)
indice_fold = 1

res_precision = []
res_recall= []
f_score=[]

for trainset, testset in kf.split(data):
    print('='*50)
    print ('FOLD # ' + str(indice_fold))
    print('='*50)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

    avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
    avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
    f=F_score(avg_precision,avg_recall)
    
    print (avg_precision)
    print (avg_recall)
    print(f)
    
    res_precision.append(avg_precision)
    res_recall.append(avg_recall)
    f_score.append(f)
    indice_fold += 1
    
    
print('='*50)
print ('MOYENNES')
print('='*50)
print('précision@k moyenne: ' + str(np.mean(res_precision)))
print('rappel@k moyen: ' + str(np.mean(res_recall)))
print('F-score moyen: ' + str(np.mean(f_score)))


FOLD # 1
1.0
0.0069605568445475635
0.013824884792626725
FOLD # 2
1.0
0.006792577866136514
0.01349350008227744
FOLD # 3
1.0
0.007953603976801988
0.015781686667762616
FOLD # 4
1.0
0.009283819628647215
0.01839684625492773
FOLD # 5
1.0
0.010107705053852526
0.02001312335958005
MOYENNES
précision@k moyenne: 1.0
rappel@k moyen: 0.008219652673997161
F-score moyen: 0.016302008231434913
